In [1]:
import datetime
print('Last Updated On: ', datetime.datetime.now())
print('Author: Philipe Bujold')

Last Updated On:  2022-09-26 14:34:59.260113
Author: Philipe Bujold


In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import seaborn as sb 
cm = sb.diverging_palette(5, 250, as_cmap=True)

# import surveyAnalysis as survey #custom analysis scripts
import pyreadstat
import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"
from plotly.subplots import make_subplots


from scipy.stats import pearsonr

In [3]:
#gather custon plotting libraries
import surveyAnalysis as survey
from surveyAnalysis import gatherScore_fromString, plot_categories, gatherFactors, compareConditions_2, plot_single_proportions_2

In [4]:
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False, connected=True)
import itables.options as opt
opt.column_filters = "footer"
opt.lengthMenu = [5, 10, 20, 50, 100, 200, 500]

c:\Anaconda\envs\pandas-profiling\lib\site-packages\itables\javascript.py:62: UserWarning:

Did you know? init_notebook_mode(all_interactive=False, connected=True) does nothing. Feel free to remove this line, or pass warn_if_call_is_superfluous=False.



In [5]:
#import sav data and weights for data

iDTC, meta = pyreadstat.read_sav(r"C:\Users\pbujold\OneDrive - Rare\Projects\2022 - CC iDTC Survey Sprint\Analysis\spss.sav")
iDTC_enriched = pyreadstat.set_value_labels(iDTC, meta, formats_as_category=True, formats_as_ordered_category=True)
iDTC_enriched.head(3)

weighted = pd.read_csv(r'C:\Users\pbujold\OneDrive - Rare\Projects\2022 - CC iDTC Survey Sprint\Analysis\weighted_data.csv')
weighted.Vrid = weighted.Vrid.astype('string')
iDTC_enriched = pd.merge(iDTC_enriched, weighted[['ethnicity', 'wt', 'Vrid', 'AGE(1)', 'GENDER(2)', 'ZIP(3)', 'REGION(4)',
       'HISPANIC(5)', 'age_group',	'SASSY']], how='left',  on='Vrid')

#remove respondents that don't have weights as their repsonses were not recorded properly
iDTC_enriched = iDTC_enriched.loc[~iDTC_enriched.isna()['wt']]

c:\Anaconda\envs\pandas-profiling\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning:

Columns (5,168,233) have mixed types.Specify dtype option on import or set low_memory=False.



In [6]:
#Filter for completes and ages that make sense
print("Original dataframe lenght: " + str(len(iDTC_enriched)))

iDTC_enriched = iDTC_enriched.loc[(iDTC_enriched['Vstatus'] == 'Complete') & (18 <= iDTC_enriched['age']) & (150 >= iDTC_enriched['age'])]
iDTC_enriched.drop(columns = ["Vcid",
                        "Vcomment",
                        "Vlanguage",
                        "Vreferer",
                        "Vsessionid",
                        "Vuseragent",
                        "Vip",
                        "Vlong",
                        "Vlat"], inplace = True)

print("...")
print("Lenght after filtering: " + str(len(iDTC_enriched)))

Original dataframe lenght: 4180
...
Lenght after filtering: 4180


In [7]:
#remove order for categories that have none
for col in ['VGeoRegion', 'var193', 'ethnicity', "education", "income", "marital_status", 'age_group', 'SASSY', 'sex']:
    iDTC_enriched[col] = iDTC_enriched[col].astype("category").cat.as_unordered()

#add order for categories that have one
for col in ["education", "income", 'age_group', 'SASSY']:
    iDTC_enriched[col] = iDTC_enriched[col].cat.as_ordered()

#recategorize continuous variables
iDTC_enriched['politics'] = pd.cut(iDTC_enriched['politics'].astype('float'), bins=[0, 20, 40, 60, 80, 100],
                    labels=['Very left', 'Left', 'Center',
                            'Right', 'Very Right'])

In [8]:
#Rename the A/B condition colum to make it easier to split between conditons for analysis 

rename_socialMedia = {'var428' : 'A/B condition'}

#rename iDTC data
iDTC.rename(columns = rename_socialMedia, inplace = True)
iDTC_enriched.rename(columns = rename_socialMedia, inplace = True)

#rename meta data labels and values
for k, v in rename_socialMedia.items():
    meta.column_names_to_labels[v] = meta.column_names_to_labels.pop(k)
    try:
        meta.variable_value_labels[v] = meta.variable_value_labels.pop(k)
    except:
        continue

In [9]:
# Create simpler variable for the experimental split

iDTC_enriched['EV treatment'] = ["Rental" if "Rental" in col else "Purchase" for col in iDTC_enriched['A/B condition']]
iDTC_enriched['Energy treatment'] = ["Green" if "Green" in col else "Solar" for col in iDTC_enriched['A/B condition']]

In [10]:
#Reorder the yes/no answers that are wrong
for col in meta.variable_value_labels.keys():
    if list(meta.variable_value_labels[col].values()) == ['Yes', 'No']:
        # print("reordered question:" + "'" + meta.column_names_to_labels[col] + "'")
        iDTC_enriched[col] = iDTC_enriched[col].cat.reorder_categories(['No', 'Yes'])
        # print(iDTC_enriched[col].cat.categories)

In [11]:
rename_columns = { 'var200O1398': 'smUse_facebook',
 'var200O1399': 'smUse_instagram',
 'var200O1400': 'smUse_pinterest',
 'var200O1401': 'smUse_reddit',
 'var200O1402': 'smUse_snapchat',
 'var200O1403': 'smUse_tiktok',
 'var200O1404': 'smUse_twitter',
 'var200O1405': 'smUse_youtube',
 'var200O1406': 'smUse_other',
 'var200O2786': 'smUse_none',
 'var201O1407': 'smDaily_facebook',
 'var201O1408': 'smDaily_instagram',
 'var201O1409': 'smDaily_pinterest',
 'var201O1410': 'smDaily_reddit',
 'var201O1411': 'smDaily_snapchat',
 'var201O1412': 'smDaily_tiktok',
 'var201O1413': 'smDaily_twitter',
 'var201O1414': 'smDaily_youtube',
 'var201O1415': 'smDaily_other',
 'var201O2787': 'smDaily_none',
 'var386': 'smInterest_health',
 'var387': 'smInterest_finance',
 'var388': 'smInterest_weather',
 'var389': 'smInterest_tech',
 'var390': 'smInterest_climate',
 'var391': 'smInterest_food',
 'var392': 'smInterest_lifestyle',
 'var212O1432': 'smPost_facebook',
 'var212O1433': 'smPost_instagram',
 'var212O1434': 'smPost_pinterest',
 'var212O1435': 'smPost_reddit',
 'var212O1436': 'smPost_snapchat',
 'var212O1437': 'smPost_tiktok',
 'var212O1438': 'smPost_twitter',
 'var212O1439': 'smPost_youtube',
 'var212O1440': 'smPost_other',
 'var212O2788': 'smPost_none',
 'var394': 'smPositive_health',
 'var395': 'smPositive_finance',
 'var396': 'smPositive_weather',
 'var397': 'smPositive_tech',
 'var398': 'smPositive_climate',
 'var399': 'smPositive_food',
 'var400': 'smPositive_lifestyle',
 'var410': 'smNegative_health',
 'var411': 'smNegative_finance',
 'var412': 'smNegative_weather',
 'var413': 'smNegative_tech',
 'var414': 'smNegative_climate',
 'var415': 'smNegative_food',
 'var416': 'smNegative_lifestyle',
 'var418': 'smLikely_health',
 'var419': 'smLikely_finance',
 'var420': 'smLikely_weather',
 'var421': 'smLikely_tech',
 'var422': 'smLikely_climate',
 'var423': 'smLikely_food',
 'var424': 'smLikely_lifestyle',
 'var217': 'waste_adp', #'Are you actively trying to reduce your food waste?',
 'var218': 'waste_adp_steps', #'Can you tell us a bit more about the steps you are taking to reduce food waste?',
 'var220': 'wasteSteps_list', #'I make a grocery list when I go grocery shopping:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var221': 'wasteSteps_expire',
 'var222': 'wasteSteps_favail', # 'I adjust plans for what I am cooking based on what I have in the fridge:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var223': "wasteSteps_compost", #'I compost food scraps / other food waste instead of putting it in the garbage:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var224': "wasteSteps_fcheck", # 'I check my fridge and pantry before buying food:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var225': "wasteSteps_ffull", #'I pack my fridge to the brim:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var226': "wasteSteps_usage", #'I make a point to use up food that is about to expire:We all do different things when it comes to shopping for food and cooking  How often do you do the following behaviors?',
 'var900': "wasteAtt_sust",# 'It is more environmentally sustainable to waste less food:How much do you agree or disagree with the following statement?',
 'var9001': "wasteAtt_clim",#'It is important to avoid wasting food to help alleviate climate change:How much do you agree or disagree with the following statement?',
 'var9002': "wasteAtt_cash",#'It saves me money to waste less food:How much do you agree or disagree with the following statement?'}
 'var304': "energy_solar_payFor" ,# 'Does your household pay for their own electricity?',
 'var305': "energy_solar_choice" ,# 'Does your electric utility give you the option to choose which electricity provider you use?',
#  'var312': "energy_green_resp" ,# 'The responsibility to make the switch to renewable energy belongs with governments and utilities, not people like me:For this next question, suppose your community was trying to increase the share of renewable energy on its power grid  How much do you agr',
 'var311': "energy_solar_oePurchase" ,# 'If I buy solar energy from an energy provider or my power utility, my choice will increase the amount of renewable energy on the local power grid:For this next question, suppose your community was trying to increase the share of renewable energy on its po',
 'var314': "energy_solar_oeInstall" ,# 'If I install solar panels on my home and allow their energy to flow onto the grid, my choice will increase the amount of renewable energy on the local power grid:For this next question, suppose your community was trying to increase the share of renewable',
 'var313': "energy_solar_resp" ,# 'The responsibility to install solar panels belongs with governments and utilities, not people like me:For this next question, suppose your community was trying to increase the share of renewable energy on its power grid  How much do you agree or disagree',
 'var318': "energy_solar_consider" ,# 'Before taking this survey, had you considered using community solar in the last 12 months?',
 'var320': "energy_solar_adp" ,# 'Do you currently get your electricity from community solar?',
 'var321': "energy_solar_dNorm" ,# 'Imagine 10 households you know  If you had to guess, how many of them do you think get their electricity from community solar?',
 'var322': "energy_solar_pNorm" ,# 'If people have the choice, should they choose community solar because it is the right thing to do?',
 'var323': "energy_solar_iNorm" ,# 'Imagine 10 people you know  If you had to guess, how many think that people should choose community solar because it is the right thing to do?',
#  'var359': "att_check_energy" ,# 'This question makes sure you are reading along  Please select three on this question',
 'var425': "energy_solar_cost" ,# 'If you had to guess, how do you think the cost of electricity from community solar compares to the cost of a traditional energy provider? (-50% meaning 50% less than a traditional energy provider, 50% meaning 50% more than a traditional energy provider)',
 'var330': "energy_solar_intent" ,# 'How likely is it that you will have community solar in the next 12 months?',
 'var331': "energy_solar_cBenefit" ,# ' How much do you think having community solar would benefit your community?',
 'var332': "energy_solar_pBenefit" ,# 'How much do you think having community solar would benefit you personally?',
 'var195': "energy_solar_efficacy" ,# 'How confident are you that you can subscribe to community solar for your home or apartment?',
 'var333': "energy_solar_interest" ,# 'How interested would you be in participating in a program which helps you sign up for community solar?',}
 'var335': "energy_green_payFor", #'Does your household pay for their own electricity?',
 'var336': "energy_green_choice", #'Does your electric utility give you the option to choose which electricity provider you use?',
#  'var338': "energy_green_resp", #'The responsibility to make the switch to renewable energy belongs with governments and utilities, not people like me:For this next question, suppose your community was trying to increase the share of renewable energy on its power grid  How much do you agr',
 'var339': "energy_green_oePurchase", #'If I buy green energy from an energy provider or my power utility, my choice will increase the amount of renewable energy on the local power grid:For this next question, suppose your community was trying to increase the share of renewable energy on its po',
 'var343': "energy_green_consider", #'Before taking this survey, had you considered using green energy in the last 12 months?',
 'var344': "energy_green_adp", #'Do you currently get your electricity from a green energy provider?',
 'var345': "energy_green_dNorm", #'Imagine 10 households you know  If you had to guess, how many of them do you think get their electricity from a green energy provider?',
 'var346': "energy_green_pNorm", #'If people have the choice, should they choose a green energy provider because it is the right thing to do?',
 'var347': "energy_green_iNorm", #'Imagine 10 people you know  If you had to guess, how many think that people should choose a green energy provider because it is the right thing to do?',
#  'var360': "att_check_energy", #'This question makes sure you are reading along  Please select three on this question',
 'var426': "energy_green_cost", #'If you had to guess, how do you think the cost of electricity from a green energy provider compares to the cost of a traditional energy provider? (-50% meaning 50% less than a traditional energy provider, 50% meaning 50% more than a traditional energy pro',
 'var349': "energy_green_intent", #'How likely is it that you will have a green energy provider in the next 12 months?',
 'var350': "energy_green_cBenefit", #' How much do you think having a green energy provider would benefit your community?',
 'var351': "energy_green_pBenefit", #'How much do you think having a green energy provider would benefit you personally?',
 'var352': "energy_green_efficacy", #'How confident are you that you can subscribe to a green energy provider for your home or apartment?',
 'var353': "energy_green_interest", #'How interested would you be in participating in a program which helps you sign up for green energy?',
 'var361': "ev_rentalAtt_quantity", #'In the last year, how many separate times have you rented a car? Car rentals can be daily or hourly rentals',
 'var362': "ev_rentalAtt_reason", #'Considering the car rentals you made in the last year, what was the primary purpose of your car rental?',
#  'var362O2240Othr': 'Something else - Please specify::Considering the car rentals you made in the last year, what was the primary purpose of your car rental?',
 'var364': "ev_rentalAtt_duration", # Thinking back to your car rentals from last year: how long was your typical car rental?',
# 'var364O2246Othr': '1 week:Thinking back to your car rentals from last year: how long was your typical car rental?',
#  'var365': "ev_rentalAtt_priority", #'What is most important to you when choosing a rental car?',
#  'var365O2257Othr': 'Something else - Please specify::What is most important to you when choosing a rental car?',
#  'var452': "ev_rentalAtt_priority", #'What do you think would be the most important to you if you were to choose a rental car?',
#  'var452O2784Othr': 'Something else - Please specify::What do you think would be the most important to you if you were to choose a rental car?',
 'var367': "ev_rentalInd_adp", #'In the last year, how many times have you rented a fully electric car? Car rentals can be daily or hourly rentals',
 'var369': "ev_rentalInd_consider", #'Before taking this survey, have you ever considered renting a fully electric car? Car rentals can be daily or hourly rentals',
 'var370': "ev_rentalInd_avail", #'Are fully electric cars available to rent in your community?',
 'var371': "ev_rentalInd_dNorm", #'Imagine 10 people you know who rent cars  If you had to guess, how many of them do you think rent fully electric cars?',
 'var372': "ev_rentalInd_pNorm", #'Do you think that people who rent cars should rent fully electric cars because it is the right thing to do?',
 'var374': "ev_rentalInd_iNorm", #'Imagine 10 people you know  If you had to guess, how many of them think that people who rent cars should rent fully electric cars because it is the right thing to do?',
 'var375': "ev_rentalAtt_range", #'What is the minimum driving range you would want a fully electric car to have in order to consider renting it?',
 'var376': "ev_rentalAtt_chargeSpot", #'Where do you think people who rent fully electric vehicles do the majority of their vehicle charging?',
#  'var376O2327Othr': 'Other - Please specify::Where do you think people who rent fully electric vehicles do the majority of their vehicle charging?',
 'var377': "ev_rentalInd_cost" , #'If you had to guess, how do you think the rental price of a fully electric car compares to the rental price of a comparable gas car? (-50% meaning 50% less than the rental price of a comparable gas car, 50% meaning 50% more than a comparable gas car)',
 'var378': "ev_rentalInd_cBenefit", #'How much do you think that when renting a car, renting a fully electric car would benefit your community?',
 'var379': "ev_rentalInd_pBenefit", #'How much do you think that when renting a car, renting a fully electric car would benefit you, personally?',
 'var380': "ev_rentalInd_efficacy", #'How confident are you in your ability to rent and drive a fully electric rental car?',
 'var382': "ev_rentalInd_interest", #'How interested would you be in participating in a program which helps you rent a fully electric car?',
 'var458': "ev_purchaseAtt_quantity",# 'How many cars does your household have?',
 'var456': "ev_purchaseInd_adp", #'A fully electric car is one that runs entirely on electricity and does not use any gasoline  How many of the cars your household has are fully electric?',
 'var435': "ev_purchaseInd_consider", #'Before taking this survey, have you ever considered getting a fully electric car?',
 'var448': "ev_purchaseInd_dNorm", #'Imagine 10 drivers you know  If you had to guess, how many of them do you think drive a fully electric car?',
 'var438': "ev_purchaseInd_pNorm", #'Do you think that people should drive a fully electric car because it is the right thing to do?',
 'var450': "ev_purchaseInd_iNorm", #'Imagine 10 people you know  If you had to guess, how many of them think that people should drive a fully electric car because it is the right thing to do?',
 'var440': "ev_purchaseAtt_range", #'What is the minimum miles of range you would want a fully electric car to have to consider purchasing it?',
 'var441': "ev_purchaseAtt_chargeSpot", #'Where do you think drivers of fully electric cars do the majority of their vehicle charging?',
#  'var441O2711Othr': 'Other - Please specify::Where do you think drivers of fully electric cars do the majority of their vehicle charging?',
 'var442': "ev_purchaseInd_cost", #'For this question, the total cost of ownership of a car includes the purchase price after any rebates or subsidies, plus the cost of maintenance and gas or power for 10 years  If you had to guess, how do you think the total cost of owning a fully electric',
 'var443': "ev_purchaseInd_cBenefit", #'How much do you think you driving a fully electric car would benefit your community?',
 'var444': "ev_purchaseInd_pBenefit", #'How much do you think purchasing and driving a fully electric car would benefit you personally?',
 'var445': "ev_purchaseInd_efficacy", #'How confident are you in your ability to purchase and drive a fully electric car?',
 'var446': "ev_purchaseInd_interest", #'How interested would you be in participating in a program which helps you purchase a fully electric car?',
 'pbd_adoption_meals': "pbd_stats_mealThree", # 'Think about the last three meals you ate  How many of these meals included meat?',
 'pbd_adoption_total': "pbd_stats_mealTotals", # 'If you were to guess, what percent of your meals include meat?',
 'pbd_attempt': "pbd_adp_intention" , #Are you trying to limit your meat consumption?',
 'var237': "pbd_adp_why", #'Why are you trying to reduce/limit your meat consumption?',
 'var238': "pbd_adp_steps", #'Can you tell us a bit more about the steps you are taking to reduce/limit your meat consumption?',
 'var259': "pbd_reason_adp", #'I am eating less meat than I was 5 years ago',
 'var9003': "pbd_reason_money", #'I am eating less meat to save money:How much do you agree or disagree with the following statement?',
 'var9004': "pbd_reason_env", #'I am eating less meat to help the environment:How much do you agree or disagree with the following statement?',
 'var9005': "pbd_reason_health", #'I am eating less meat to help my personal health:How much do you agree or disagree with the following statement?',
 'var9006': "pbd_reason_peta", #'I am eating less meat to help animal welfare:How much do you agree or disagree with the following statement?',
#  'var9007': "pbd_reason_dNorm", #'It is becoming more commonplace to eat less meat:How much do you agree or disagree with the following statement?',
#  'var9008': "pbd_reason_iNorm", #'People think it’s good to reduce meat consumption:How much do you agree or disagree with the following statement?',
# CHECK THE NEXT TWO, SOMETHING WEIRD GOING ON
#  'var255': 'People think it’s good to reduce meat consumption:How much do you agree or disagree with the following statement?',
#  'var254': 'It is becoming more commonplace to eat less meat:How much do you agree or disagree with the following statement?',
 'var260': "pbd_adp_sharing", #'You noted that you are trying to eat less or no meat  Is this something you’ve shared with others?',
 'var261': "pbd_adp_sharingWhy", #"Why have you shared, or why haven't you shared with others?",
 'var427O2618': "pbd_sharing_family", #'Your family:Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
 'var427O2619': "pbd_sharing_friends", #'Your close friends:Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
 'var427O2620': "pbd_sharing_colleagues", #'Your colleagues:Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
 'var427O2621': "pbd_sharing_neighbors", #'Your neighbors:Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
 'var427O2622': "pbd_sharing_socialMedia", #'Your social media followers:Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
#  FIX THE NEXT TWO
#  'var427O2623': 'Other - Write In (Required):Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
#  'var427O2785': 'Other - Please specify::Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
#  'var427O2785Othr': 'Other - Please specify::Of the following groups of people in your life, which ones do you think know that you are trying to eat less meat?',
 # Fix the next ones so that they merge properly
#  'var264': "pbd_social_approval", #'Imagine 10 people you know, If you had to guess, how many of them approve of you eating less meat?',
#  'var266': "pbd_social_approval", #'Imagine 10 people you know, If you had to guess, how many of them would approve if you started eating less meat?',
#  'var265': "pbd_social_disapproval", #'Imagine 10 people you know, If you had to guess, how many of them disapprove of you eating less meat?',
#  'var267': "pbd_social_disapproval", #'Imagine 10 people you know, If you had to guess, how many of them would disapprove if you started eating less meat?',
 'var268': "pbd_social_notSharing", # 'Are there any people or groups of people in your life you think feel strongly against eating less (or no) meat?',
 'var269': "pbd_social_notSharing_who", #'Please tell us who or which groups of people that would that be?',
 'var284': "pbd_interest_health", #'I am interested in learning how reducing my meat consumption can help my health and wellbeing:How much do you agree or disagree with the following statement?',
 'var285': "pbd_interest_env", #'I am interested in learning how reducing my meat consumption can benefit the environment:How much do you agree or disagree with the following statement?',
 'var286': "pbd_interest_money", #'I am interested in learning how reducing my meat consumption can benefit me financially:How much do you agree or disagree with the following statement?',
 'var287': "pbd_interest_orgHealth", #'I think positively about organizations that champion healthy food:How much do you agree or disagree with the following statement?',
 'var288': "pbd_interest_orgEnv", #'I think positively about organizations that champion environmentally sustainable food:How much do you agree or disagree with the following statement?',
 'var289': "pbd_adp_roleModel", #"Who's guidance/tips would you prefer to receive about reducing your meat consumption?",
#  'var289O1890Othr': "None of the above:Who's guidance/tips would you prefer to receive about reducing your meat consumption?",
 'var290': "pbd_adp_roleModel_why", #'Why would you prefer to receive advice from the group you selected?',
 'var297': "pbd_attitudes_health", #'I think a lot about my health when it comes to my food choices:How much do you agree or disagree with the following statements?',
 'var302': "pbd_attitudes_money", #'I think a lot about cost when it comes to my food choices:How much do you agree or disagree with the following statements?',
 'var299': "pbd_attitudes_env", #'I think a lot about environmental sustainability when it comes to my food choices:How much do you agree or disagree with the following statements?',
 'var300': "pbd_attitudes_envChoice", #'Eating less meat is a more environmentally sustainable choice:How much do you agree or disagree with the following statements?',
 'var298': "pbd_attitude_healthChoice", #'It is good for my health to eat less meat:How much do you agree or disagree with the following statements?',
 'var303': "pbd_attitude_moneyChoice", #'It saves me money to eat less meat:How much do you agree or disagree with the following statements?',
 'var301': "pbd_attitude_cc", #'It is important to eat less meat to help alleviate climate change:How much do you agree or disagree with the following statements?', }
}

#rename iDTC data
iDTC.rename(columns = rename_columns, inplace = True)
iDTC_enriched.rename(columns = rename_columns, inplace = True)

#rename meta data labels and values
for k, v in rename_columns.items():
    meta.column_names_to_labels[v] = meta.column_names_to_labels.pop(k)
    try:
        meta.variable_value_labels[v] = meta.variable_value_labels.pop(k)
    except:
        continue


meta.column_names_to_labels['pbd_reason_dNorm'] = meta.column_names_to_labels.pop('var9007')
meta.column_names_to_labels['pbd_reason_iNorm'] = meta.column_names_to_labels.pop('var9008')

meta.column_names_to_labels['pbd_social_approval'] = 'Imagine 10 people you know, If you had to guess, how many of them approve/would approve of you eating less meat?'
meta.column_names_to_labels['pbd_social_disapproval'] = 'Imagine 10 people you know, If you had to guess, how many of them disapprove/would dissaprove of you eating less meat?'

In [12]:
# need to change the nans in ev_rentalInd_adp to 0 if ev_rentalAtt_quantity is 0
# same with ev_purchaseInd_adp and ev_purchaseAtt_quantity

iDTC_enriched['ev_rentalAtt_priority'] = np.nan
iDTC_enriched.loc[iDTC_enriched['var365'].notnull(), ['ev_rentalAtt_priority']] =  iDTC_enriched['var365'].loc[iDTC_enriched['var365'].notnull()]
iDTC_enriched.loc[iDTC_enriched['var452'].notnull(), ['ev_rentalAtt_priority']] =  iDTC_enriched['var452'].loc[iDTC_enriched['var452'].notnull()]
iDTC_enriched['ev_rentalAtt_priority'] = iDTC_enriched['ev_rentalAtt_priority'].astype('category')

meta.column_names_to_labels['ev_rentalAtt_priority'] = meta.column_names_to_labels['var365']

iDTC_enriched['ev_rentalAtt_adp'] = iDTC_enriched['ev_rentalInd_adp'].values
iDTC_enriched['ev_rentalInd_adp'] = iDTC_enriched['ev_rentalInd_adp'].cat.codes >= 3
iDTC_enriched['ev_rentalInd_adp'] = iDTC_enriched['ev_rentalInd_adp'].astype('category').cat.as_ordered()

iDTC_enriched['ev_rentalAtt_quantity'] = iDTC_enriched['ev_rentalAtt_quantity'].cat.reorder_categories(['None', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10+'])

iDTC_enriched['ev_rentalInd_hasPotential'] = iDTC_enriched['ev_rentalAtt_quantity'].cat.codes > 0
iDTC_enriched['ev_rentalInd_hasPotential'] = iDTC_enriched['ev_rentalInd_hasPotential'].astype('category').cat.as_ordered()
iDTC_enriched.loc[iDTC_enriched['ev_rentalAtt_quantity'].cat.codes == -1, 'ev_rentalInd_hasPotential'] = np.nan
meta.column_names_to_labels['ev_rentalInd_hasPotential'] = 'Respondent rents car(s)'

iDTC_enriched['ev_purchaseInd_hasPotential'] = iDTC_enriched['ev_purchaseAtt_quantity'].cat.codes > 0
iDTC_enriched['ev_purchaseInd_hasPotential'] = iDTC_enriched['ev_purchaseInd_hasPotential'].astype('category').cat.as_ordered()
iDTC_enriched.loc[iDTC_enriched['ev_purchaseAtt_quantity'].cat.codes == -1, 'ev_purchaseInd_hasPotential'] = np.nan
meta.column_names_to_labels['ev_purchaseInd_hasPotential'] = 'Respondent owns car(s)'

In [13]:
#merge and recreate category for social approval - both conditions had this question
iDTC_enriched['pbd_social_approval'] = iDTC_enriched['var264'].astype(str).replace('nan', '') + iDTC_enriched['var266'].astype(str).replace('nan', '')
iDTC_enriched['pbd_social_approval'] = iDTC_enriched['pbd_social_approval'].astype("category")
iDTC_enriched['pbd_social_approval'].cat.categories = iDTC_enriched['var264'].cat.categories
iDTC_enriched['pbd_social_approval'] = iDTC_enriched['pbd_social_approval'].cat.as_ordered()


iDTC_enriched['pbd_social_disapproval'] = iDTC_enriched['var265'].astype(str).replace('nan', '') + iDTC_enriched['var267'].astype(str).replace('nan', '')
iDTC_enriched['pbd_social_disapproval'] = iDTC_enriched['pbd_social_disapproval'].astype("category")
iDTC_enriched['pbd_social_disapproval'].cat.categories = iDTC_enriched['var265'].cat.categories
iDTC_enriched['pbd_social_disapproval'] = iDTC_enriched['pbd_social_disapproval'].cat.as_ordered()
# iDTC_enriched['pbd_social_disapproval']

iDTC_enriched['pbd_reason_iNorm'] = iDTC_enriched['var9008'].astype(str).replace('nan', '') + iDTC_enriched['var255'].astype(str).replace('nan', '')
iDTC_enriched['pbd_reason_iNorm'] = iDTC_enriched['pbd_reason_iNorm'].astype("category")
iDTC_enriched['pbd_reason_iNorm'].cat.categories = iDTC_enriched['var9008'].cat.categories
iDTC_enriched['pbd_reason_iNorm'] = iDTC_enriched['pbd_reason_iNorm'].cat.as_ordered()

iDTC_enriched['pbd_reason_dNorm'] = iDTC_enriched['var9007'].astype(str).replace('nan', '') + iDTC_enriched['var254'].astype(str).replace('nan', '')
iDTC_enriched['pbd_reason_dNorm'] = iDTC_enriched['pbd_reason_dNorm'].astype("category")
iDTC_enriched['pbd_reason_dNorm'].cat.categories = iDTC_enriched['var9007'].cat.categories
iDTC_enriched['pbd_reason_dNorm'] = iDTC_enriched['pbd_reason_dNorm'].cat.as_ordered()

In [14]:
iDTC_enriched['ev_rentalAtt_reason'] = iDTC_enriched['ev_rentalAtt_reason'].cat.as_unordered()
iDTC_enriched['ev_rentalAtt_chargeSpot'] = iDTC_enriched['ev_rentalAtt_chargeSpot'].cat.as_unordered()
iDTC_enriched['ev_purchaseAtt_chargeSpot'] = iDTC_enriched['ev_purchaseAtt_chargeSpot'].cat.as_unordered()

In [15]:
iDTC_enriched['state'] = iDTC_enriched['state'].cat.as_unordered()

state_2_region = {"Pacific" : ['Oregon', 'Washington', 'California'],
"Rocky Mountains" : ['Colorado', 'Nevada',  'Montana', 'Utah',  'Idaho', 'Wyoming'],
"Southwest" : ['Arizona', 'New Mexico', 'Texas', 'Oklahoma' ],
"Northeast" : ['New Jersey', 'Pennsylvania', 'Vermont', 'New Hampshire','New York' , 'Massachusetts', 'Maine', 'Connecticut', 'Rhode Island'],
"Southeast" : ['Virginia','Arkansas','Louisiana', 'Mississippi', 'North Carolina', 'West Virginia', 'Alabama', 'South Carolina', 'Kentucky', 'Washington, D C ', 'Florida', 'Tennessee', 'Georgia', 'Maryland', 'Delaware'],
"Midwest" : ['Ohio', 'Missouri', 'Kansas' , 'Indiana', 'Nebraska', 'Michigan', 'Illinois', 'South Dakota', 'Minnesota', 'Iowa', 'North Dakota', 'Wisconsin'],
"Noncontiguous" : ['Hawaii','Alaska']}

newdict = {}
for key, value in state_2_region.items():
    for string in value:
        newdict.setdefault(string, key)
state_2_region = newdict
iDTC_enriched['region'] = iDTC_enriched['state'].apply(lambda x : state_2_region[x]).astype('category')

-----
## Energy (Community Solar vs Green Electricity scores) <a name="fourth-bullet"></a>

To understand if the national index values for buying green energy also apply to community solar

**What we want to be able to say**
* What are people's perception of the efficacy of community solar, and whether it's their responsibility to adopt
* Are the values between green energy at large and community solar effectively the same?
* Where do they diverge?
* Is studying one a reasonable proxy for the other?


In [16]:
#merge and recreate category for energies - both conditions had this question
iDTC_enriched['energy_green_resp'] = iDTC_enriched['var338'].astype(str).replace('nan', '') + iDTC_enriched['var312'].astype(str).replace('nan', '')
iDTC_enriched['energy_green_resp'] = iDTC_enriched['energy_green_resp'].astype("category")
iDTC_enriched['energy_green_resp'].cat.categories = iDTC_enriched['var312'].cat.categories

meta.column_names_to_labels['energy_green_resp'] = meta.column_names_to_labels.pop('var338')
# meta.column_names_to_labels['energy_green_resp'] = meta.column_names_to_labels.pop('var312')

#### Green energy
First we begin with having a look at participant responses for the **Green Electricity** questions 

In [25]:
indexer = [col for col in iDTC_enriched.columns if ("energy_green" in col)]
_ = survey.plot_single_proportions_2(
    iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Green', indexer],
    meta,
    title="Green electricity questions",
    orientation="v",
    showlegend=False,
    individual_plots = True,
)

c:\Anaconda\envs\pandas-profiling\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



Now we look at how demographics influence these results

In [23]:
indexer = [col for col in iDTC_enriched.columns if ( ("energy_green" in col) )]#| ("sm" in col) | ("waste" in col) )  | ("pbd" in col) | ("energy" in col)  ]
# indexer = [col for col in indexer if col not in longForm]

demographics = np.array(['region', 'var193', 'age_group', 'sex', 'gender','ethnicity', 'hispanic', 'hispanic_other',
        'education', 'income', 'SASSY', 'marital_status',
       'partner', 'politics', 'employment_status', 'employment_industry',
       'enviro_donation'])

corr_m = survey.plot_corrHeatMap(df = iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Green'],
                        meta =  meta,
                        x_cols =  demographics,
                        y_cols = indexer)

#### Now onto the **Community Solar** questions 

In [26]:
indexer = [col for col in iDTC_enriched.columns if ("energy_solar" in col)]
_ = survey.plot_single_proportions_2(
    iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Solar', indexer],
    meta,
    title="Community solar questions",
    orientation="v",
    showlegend=False,
    individual_plots = True,
)

Now onto how demographics affect solar energy metrics

In [27]:
indexer = [col for col in iDTC_enriched.columns if ( ("energy_solar" in col) )]#| ("sm" in col) | ("waste" in col) )  | ("pbd" in col) | ("energy" in col)  ]
# indexer = [col for col in indexer if col not in longForm]

demographics = np.array(['region', 'var193', 'age_group', 'sex', 'gender','ethnicity', 'hispanic', 'hispanic_other',
        'education', 'income', 'SASSY', 'marital_status',
       'partner', 'politics', 'employment_status', 'employment_industry',
       'enviro_donation'])

corr_m = survey.plot_corrHeatMap(df = iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Solar'],
                        meta =  meta,
                        x_cols =  demographics,
                        y_cols = indexer)

### Index Metric Comparison

Below you can see a comparison of normalized, weighted averages for each question that was asked on the topic of energy. Since we wanted to know whether answers in the **Community Solar** condition were equivalemnt to those in the **Green Electricity** condition, we set a bound of +/- 10% on the green electricity average and contrasted the community solar average to it. A significant difference therefore represents a difference in %responses greater than +/- 10%. 

In [19]:
SolarResults, _ = gatherScore_fromString(iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Solar'], meta, ['energy_solar'], replaceCols=False,
                                            )
GreenResults, _ = gatherScore_fromString(iDTC_enriched.loc[iDTC_enriched['Energy treatment'] == 'Green'], meta, ['energy_green'], replaceCols=False,
                                            )
energyDF = pd.concat((SolarResults, GreenResults))
energyDF.rename(index = {'energy_solar_avg' : 'Community Solar',
                                 'energy_green_avg' : 'Green energy'},
                                columns= {"adp" : "Adoption",
                                        "payFor" : "Pay for own utility",
                                	    "choice" : "Has provider option",
                                        "oePurchase" : "Outcome efficacy from purchasing",
                                        "oeInstall"	: "Outcome efficacy from installing",
                                        "resp" : "Responsibility belongs supply-side",
                                        "consider" : "Consideration",
                                        "dNorm" : "Belief that others have adopted",
                                        "pNorm"	: "Belief that others should adopt",
                                        "iNorm" : "Belief that others think people should adopt",
                                        "cost" : "Cost relative usual",
                                        "intent" : 	"Intention",
                                        "cBenefit" : "Perceived community benefit",
                                        "pBenefit" : "Perceived personal benefit",
                                        "efficacy" : "Self-efficacy",
                                        "interest" : "Program interest"},
                                inplace = True)
display(energyDF.drop(columns=["Pay for own utility","Has provider option"]).style.background_gradient(cmap=cm, vmin = 0, vmax = 1))


,Outcome efficacy from purchasing,Outcome efficacy from installing,Responsibility belongs supply-side,Consideration,Adoption,Belief that others have adopted,Belief that others should adopt,Belief that others think people should adopt,Cost relative usual,Intention,Perceived community benefit,Perceived personal benefit,Self-efficacy,Program interest
Community Solar,0.587223,0.651333,0.432768,0.239870,0.681163,0.169560,0.717196,0.424053,0.394500,0.245110,0.591282,0.549406,0.623166,0.484483
Green energy,0.578347,nan,0.464404,0.435934,0.711376,0.212070,0.763814,0.470735,0.444963,0.310991,0.599015,0.551799,0.572353,0.499770


In [20]:
# Equivalency Threshold:
threshold = .1

In [21]:

print('\n')
print("The relevant difference threshold has been set at a \033[1m " + str(threshold * 100) + "%\033[0m difference.")

df2 = survey.gatherFactors(iDTC_enriched.loc[iDTC_enriched['A/B condition'].isin(['Community Solar + Purchase', 'Community Solar + Rental'])], ['energy_solar'])
df1 = survey.gatherFactors(iDTC_enriched.loc[iDTC_enriched['A/B condition'].isin(['Green Energy + Purchase', 'Green Energy + Rental'])], ['energy_green'])
results = survey.compareConditions_2(df1, df2, threshold = threshold, names = ['Green energy','Community solar'], title = 'Energy Source Index Comparison')

results.rename( index= {"adp" : "Adoption",
                        "payFor" : "Pay for own utility",
                        "avail" : "Available in community",
                        "oePurchase" : "Outcome efficacy from purchasing",
                        "oeInstall"	: "Outcome efficacy from installing",
                        "resp" : "Responsibility belongs supply-side",
                        "consider" : "Consideration",
                        "dNorm" : "Belief that others have adopted",
                        "pNorm"	: "Belief that others should adopt",
                        "iNorm" : "Belief that others think people should adopt",
                        "cost" : "Cost relative usual",
                        "intent" : 	"Intention",
                        "cBenefit" : "Perceived community benefit",
                        "pBenefit" : "Perceived personal benefit",
                        "efficacy" : "Self-efficacy",
                        "interest" : "Program interest"},
                                inplace = True)
# results.style.background_gradient(subset=['sig'], cmap=cm, vmin = 0, vmax = 0.1)
results.drop(columns=['std1', 'std2'], inplace = True)

def color_recommend(s):
    return np.where(s>0.05,
                    'background-color: pink',
                    'background-color: lightblue')

def color_recommend2(s):
    return np.where(s>threshold or s<-threshold,
                    'color: red',
                    'color: blue')

results.style.applymap(color_recommend2, subset=['diff']).apply(color_recommend, subset=['sig'])



The relevant difference threshold has been set at a  10.0% difference.


,Green energy,Community solar,diff,outcome,sig
variable,,,,,
Adoption,0.567559,0.523424,0.044135,equivalent,0.000000
Perceived community benefit,0.599015,0.591282,0.007733,equivalent,0.000000
choice,0.450624,0.451708,-0.001084,equivalent,0.000000
Consideration,0.435934,0.239870,0.196064,non-equivalent,1.000000
Cost relative usual,0.444963,0.394500,0.050463,equivalent,0.000000
Belief that others have adopted,0.212070,0.169560,0.042510,equivalent,0.000000
Self-efficacy,0.572353,0.623166,-0.050813,equivalent,0.000001
Belief that others think people should adopt,0.470735,0.424053,0.046682,equivalent,0.000000
Intention,0.310991,0.245110,0.065882,equivalent,0.000193


With a threshold of 10%, current results indicate that respondent's only non-equivalent set of answers is for the question of whether they had considered getting green/solar energy in the last 12 months.

Specifically:
* *'Before taking this survey, had you considered using green energy in the last 12 months?'*
* *'Before taking this survey, had you considered using community solar in the last 12 months?'*